# 3. laboratorijska vježba - u izradi
### Visoko-propusni filtri

<img src="static/rl-highpass.png" alt="simple-rl-highpass" width="400"/>

Link za interaktivni rad sa pasivnim visoko-propusnim filtrom: http://sim.okawa-denshi.jp/en/CRtool.php

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.fftpack import fftshift
from scipy.signal import (butter, lfilter, freqz)

def rl_highpass(R, L, f):
    return 1 / (1 + 1/(1.0j * 2 * np.pi * f * L / R))

# configuration
R = 200.0         # Ohms
L = 100*10**(-3)   # Henry
f = np.linspace(0.1, 10000, num=10000)
f_cutoff = R/(2*np.pi*L)
print(f'Cut-off frequency = {f_cutoff}Hz')

H = rl_highpass(R, L, f)
plt.figure(num=None, figsize=(12, 6), dpi=80)
plt.plot(f*2*np.pi, [abs(h) for h in H], label='Frequency response')
plt.plot(f_cutoff*2*np.pi, 1/np.sqrt(2), marker='o', label='Cut-off point')
plt.annotate(f'   ω_c = {round(f_cutoff*2*np.pi, 2)}\n  H(ω_c)={round(1/np.sqrt(2), 2)}', 
             (f_cutoff*2*np.pi, 1/np.sqrt(2)))
plt.xscale('log')
plt.title(f'R={R/1000}kΩ\nL={L*10**3}mF')
plt.xlabel('ω [Hz]')
plt.ylabel('abs(V_out/V_in)(ω)')
plt.legend(loc='best')
plt.show()

### Strujno-naponska karakteristika

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.figure(num=None, figsize=(12, 6), dpi=80)

l = 0.0229      #Inductance (H)
r = 3.34        #Resistance (Ohm)
v = 5           #Voltage (V) DC
i = v/r         #Peak current (A)
tau = l/r       #Tau time constant
a = tau * 4.4   #critical time value at which current is switched (switching occurs every a seconds)

t = np.linspace(0,2*a,2000) #Time vector

def initialCurrent():
    current = []
    for i in t:
        if i <= a:
            current.append((v/r)*(1-np.exp((-r/l)*i)))          #I(t) = v/r * [1-exp((-r/l)*t)]
        else:
            current.append(0)
    return np.array(current)

#Plot Icurrent
Icurrent = initialCurrent()
plt.plot(t,Icurrent,label='current',color='blue')

def laterCurrent():
    current = []
    for i in t:
        if i >= a:
            current.append(Icurrent[-1001]*np.exp((-r/l)*(i-a)))  #I(t) = Ir * exp((-r/l)*(t-t0))
        else:
            current.append(0)
    return np.array(current)

#Plot current after switch off
current_off = laterCurrent()
plt.plot(t,current_off,color='blue')

#Current on and off cycle: on and off at the zeros of the it function
f = 1/(2*a)                 #frequency f = 1/T
w = np.pi/a                 #w = 2pi * f
it = i*np.sin(w*t)
plt.plot(t,it,label='On/off cycle',color='green')

#Plot zeros (switching points)
zeros = np.array([0,a,2*a])
zeros_i = i*np.sin(w*zeros)
plt.plot(zeros,zeros_i,marker='x',markersize=10,label='On and off',color='red')
plt.annotate('On',xy=(zeros[0]+0.0005,zeros_i[0]))
plt.annotate('Off',xy=(zeros[1]+0.0005,zeros_i[1]))
plt.annotate('On',xy=(zeros[2]+0.0005,zeros_i[2]))

#Critical value plot
plt.plot((a,a+0.00001),(max(it),min(it)),'r',alpha=0.9,label='Critical value')

print('Inductance (H):','\t'+str(l))
print('Resistance (Ohm):','\t'+str(r))
print('DC voltage (V):','\t'+str(v))
print('Current (A):','\t\t'+str(i))
print('Tau:','\t\t\t'+str(tau))
print('Critical time (s):','\t'+str(a))
print('\n')
print('Switch frequency (Hz):','\t'+str(f))
print('w coefficient (2*pi*f)','\t'+str(w))

#x axis line
plt.plot((0,2*a+0.0005),(0,0),'k',alpha=0.9,markersize=10)
plt.xlabel('time (s)')
plt.ylabel('current (A)')
plt.legend(loc='best')
plt.xlim(-0.002,2*a+0.003)
plt.ylim(min(it)-0.05,max(it)+0.05)

plt.show()

Following code uses the functions ```scipy.signal.firwin``` or ```scipy.signal.firwin2``` to create bandpass filters corresponding to the 128-point Hamming window with cutoff frequencies 0.7Hz and 4Hz. This assumes that the sampling is done uniformly. If the sampling is not uniform, a FIR filter is not appropriate.

Original code is taken from [here](https://stackoverflow.com/a/16306891).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from scipy.signal import freqz
from scipy.signal import firwin, remez, kaiser_atten, kaiser_beta

def bandpass_firwin(ntaps, lowcut, highcut, fs, window='hamming'):
    nyq = 0.5 * fs
    taps = firwin(ntaps, [lowcut, highcut], nyq=nyq, pass_zero=False,
                  window=window, scale=False)
    return taps

def bandpass_kaiser(ntaps, lowcut, highcut, fs, width):
    nyq = 0.5 * fs
    atten = kaiser_atten(ntaps, width / nyq)
    beta = kaiser_beta(atten)
    taps = firwin(ntaps, [lowcut, highcut], nyq=nyq, pass_zero=False,
                  window=('kaiser', beta), scale=False)
    return taps

def bandpass_remez(ntaps, lowcut, highcut, fs, width):
    delta = 0.5 * width
    edges = [0, lowcut - delta, lowcut + delta,
             highcut - delta, highcut + delta, 0.5*fs]
    taps = remez(ntaps, edges, [0, 1, 0], Hz=fs)
    return taps


# Sample rate and desired cutoff frequencies (in Hz).
fs = 63.0
lowcut = 0.7
highcut = 4.0

ntaps = 128
taps_hamming = bandpass_firwin(ntaps, lowcut, highcut, fs=fs)
taps_kaiser16 = bandpass_kaiser(ntaps, lowcut, highcut, fs=fs, width=1.6)
taps_kaiser10 = bandpass_kaiser(ntaps, lowcut, highcut, fs=fs, width=1.0)
remez_width = 1.0
taps_remez = bandpass_remez(ntaps, lowcut, highcut, fs=fs,
                            width=remez_width)

# Plot the frequency responses of the filters.
plt.figure(1, figsize=(12, 9))
plt.clf()

# First plot the desired ideal response as a green(ish) rectangle.
rect = plt.Rectangle((lowcut, 0), highcut - lowcut, 1.0,
                     facecolor="#60ff60", alpha=0.2)
plt.gca().add_patch(rect)

# Plot the frequency response of each filter.
w, h = freqz(taps_hamming, 1, worN=2000)
plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="Hamming window")

w, h = freqz(taps_kaiser16, 1, worN=2000)
plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="Kaiser window, width=1.6")

w, h = freqz(taps_kaiser10, 1, worN=2000)
plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="Kaiser window, width=1.0")

w, h = freqz(taps_remez, 1, worN=2000)
plt.plot((fs * 0.5 / np.pi) * w, abs(h),
         label="Remez algorithm, width=%.1f" % remez_width)

plt.xlim(0, 8.0)
plt.ylim(0, 1.1)
plt.grid(True)
plt.legend()
plt.xlabel('Frequency (Hz)')
plt.ylabel('Gain')
plt.title('Frequency response of several FIR filters, %d taps' % ntaps)

plt.show()